In [1]:
import pickle 
import torch 
import os 
import sys 
sys.path.append('../submodule') 
import torch_util 
from torch_util.imports import * 

with open(os.path.expanduser('~/dataset/OGB/ogbn-mag/processed/hg.dict.pkl'), 'rb') as fp: 
    graph_info = pickle.load(fp)
    
graph_info 

{'num_nodes_dict': {'author': 1134649,
  'institution': 8740,
  'paper': 736389,
  'field': 59965},
 'edge_index_dict': {('author',
   'AI',
   'institution'): tensor([[      0,       1,       2,  ..., 1134645, 1134647, 1134648],
          [    845,     996,    3197,  ...,    5189,    4668,    4668]]),
  ('author',
   'AP',
   'paper'): tensor([[      0,       0,       0,  ..., 1134647, 1134648, 1134648],
          [  19703,  289285,  311768,  ...,  657395,  671118,  719594]]),
  ('paper',
   'PP',
   'paper'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
          [    88,  27449, 121051,  ..., 421711, 427339, 439864]]),
  ('paper',
   'PF',
   'field'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
          [   145,   2215,   3205,  ...,  21458,  22283,  31934]])},
 'paper_feat': tensor([[-0.0954,  0.0408, -0.2109,  ...,  0.0616, -0.0277, -0.1338],
         [-0.1510, -0.1073, -0.2220,  ...,  0.3458, -0.0277, -0.2185],
         [-0.1148, -0.1760, 

In [2]:
edge_index_dict = graph_info['edge_index_dict']
num_nodes_dict = graph_info['num_nodes_dict']

os.makedirs('./TransE_emb', exist_ok=True)

def convert_hg_to_triplets(edge_index_dict: dict[EdgeType, EdgeIndex], 
                           num_nodes_dict: dict[NodeType, int],
                           output_path: str): 
    nid_offset_dict: dict[NodeType, int] = dict() 

    nid_offset = 0 

    for ntype, num_nodes in sorted(num_nodes_dict.items()):
        nid_offset_dict[ntype] = nid_offset 
        nid_offset += num_nodes 

    with open(output_path, 'w', encoding='utf-8') as fp:
        for etype, edge_index in tqdm(edge_index_dict.items()): 
            src_ntype, _, dest_ntype = etype 
            etype = '__'.join(etype) 
            src_nids, dest_nids = edge_index 
            
            src_nids.add_(nid_offset_dict[src_ntype])
            dest_nids.add_(nid_offset_dict[dest_ntype])

            for src_nid, dest_nid in zip(src_nids.tolist(), dest_nids.tolist()): 
                print(f"{src_nid}\t{etype}\t{dest_nid}", file=fp)

convert_hg_to_triplets(
    edge_index_dict = edge_index_dict, 
    num_nodes_dict = num_nodes_dict,
    output_path = './TransE_emb/triplets.tsv', 
)

  0%|          | 0/4 [00:00<?, ?it/s]